In [1]:
import pandas
import os
import numpy as np

data = pandas.read_csv('features_SPY.csv', delimiter=',')
data['Date'] = pandas.to_datetime(data['Date'])

X = data.iloc[:, 1:23]
#print(X)
y = np.where(data['Return_1'] >= 0, 1.0, 0.0)
#print(y)

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                      test_size=0.3, 
                                      random_state=241)

X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
print(X_train.shape)
print(X_test.shape)
print(X_train)

(1057, 22)
(453, 22)
      Return_1  Return_2  Return_3  Return_5  Return_10  Return_20  Return_40  \
0     0.019605  0.011428  0.004480 -0.006744  -0.029342  -0.018293   0.039779   
1    -0.007297 -0.032661 -0.042792 -0.036009   0.037789   0.042135  -0.113511   
2     0.003722  0.004041  0.014977  0.013921   0.063260  -0.008993   0.058730   
3    -0.013876 -0.013739 -0.030174 -0.028308  -0.019279  -0.018598   0.004486   
4    -0.006905 -0.007770 -0.014710  0.003452  -0.007914  -0.001097   0.040751   
...        ...       ...       ...       ...        ...        ...        ...   
1052 -0.006835 -0.009724 -0.030001 -0.039010  -0.017921  -0.020004  -0.013352   
1053  0.005816  0.004654  0.021806  0.024904   0.008874   0.014650   0.042225   
1054  0.008941  0.017879  0.019850  0.039477   0.009056   0.093275  -0.011183   
1055  0.003930  0.010298  0.016145  0.013189   0.024967   0.069969   0.100402   
1056  0.015237  0.018623  0.021462  0.019615   0.035899   0.040014   0.067118   

      

In [3]:
from sklearn.preprocessing import StandardScaler

#https://stackoverflow.com/questions/35723472/how-to-use-sklearn-fit-transform-with-pandas-and-return-dataframe-instead-of-num
def scaleDataFrame(dataframe, debugPrint = False):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(dataframe)
    dataframe = pandas.DataFrame(scaled_features, index=dataframe.index, columns=dataframe.columns)
    
    #check
    if debugPrint == True:
        for column in dataframe:
            print("[%s] mean = %.4f, variance = %.4f" % 
                  (column, dataframe[column].mean(), dataframe[column].var()))
    
    return dataframe

In [4]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

C = {0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0}
C = {1000.0}
#grid = {'C': np.power(10.0, np.arange(-4, 0))}
for penalty in C:
    print("trying C = " + str(penalty))
    cv = KFold(n_splits=3, shuffle=False, random_state=241)
    
    X_train = scaleDataFrame(X_train, debugPrint = True)

    for train_index, test_index in cv.split(X_train):
        #clf = svm.SVC(kernel='linear', random_state=241)
        clf = svm.SVC(C = 100000, kernel = 'linear', random_state=241)
        XX = X_train.loc[train_index, ['Return_1', 'AverageVolume_1']]
        yy = y_train[train_index]
        #print(train_index[:])
        #print(X_train[0:13])
        print(XX.iloc[:10,:])
        print(yy[:10])
        
        print("fit start")
        clf.fit(XX, yy)
        print("fit end")
        
        #clf.fit(X_train[train_index], y_train[train_index])
#gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
#gs.fit(X, y)

trying C = 1000.0
[Return_1] mean = -0.0000, variance = 1.0009
[Return_2] mean = 0.0000, variance = 1.0009
[Return_3] mean = -0.0000, variance = 1.0009
[Return_5] mean = -0.0000, variance = 1.0009
[Return_10] mean = 0.0000, variance = 1.0009
[Return_20] mean = 0.0000, variance = 1.0009
[Return_40] mean = -0.0000, variance = 1.0009
[Return_60] mean = 0.0000, variance = 1.0009
[Return_120] mean = 0.0000, variance = 1.0009
[Return_250] mean = 0.0000, variance = 1.0009
[AverageVolume_1] mean = 0.0000, variance = 1.0009
[AverageVolume_2] mean = 0.0000, variance = 1.0009
[AverageVolume_3] mean = -0.0000, variance = 1.0009
[AverageVolume_5] mean = -0.0000, variance = 1.0009
[AverageVolume_10] mean = 0.0000, variance = 1.0009
[AverageVolume_20] mean = 0.0000, variance = 1.0009
[AverageVolume_40] mean = -0.0000, variance = 1.0009
[AverageVolume_60] mean = 0.0000, variance = 1.0009
[AverageVolume_120] mean = -0.0000, variance = 1.0009
[AverageVolume_250] mean = 0.0000, variance = 1.0009
[DayOfWe